In [27]:
import praw
import pandas as pd
from bs4 import BeautifulSoup
import requests
from _secrets import user_agent, client_id, client_secret
from tqdm import tqdm

In [9]:
reddit = praw.Reddit(user_agent=user_agent,
                     client_id=client_id,
                     client_secret=client_secret)

In [10]:
gaming_wiki_content = reddit.subreddit('gaming').wiki['list-sorted-by-subscribers'].content_html
soup = BeautifulSoup(gaming_wiki_content)

In [11]:
tables = soup.find_all('table')
table = tables[0]

In [12]:
list_of_tables = [pd.read_html(str(table))[0] for table in tables]

Label tables for categorization later

In [13]:
for table_num,table in enumerate(list_of_tables):
    table['table_number'] = table_num

In [14]:
subreddits = pd.concat(list_of_tables)

In [15]:
subreddits.sample(9)

,Name,Link,Subscribers,table_number
13,Team Fortress 2,/r/tf2,146551,0
8,Fighting Games,/r/Fighters,8971,3
310,Darkfall,/r/Darkfall,1018,0
69,Rust,/r/playrust,24679,0
50,MAME,/r/MAME,5715,2
28,PlayStation 2,/r/ps2,2876,5
129,EarthBound/Mother,/r/earthbound,8386,0
2,RPG,/r/rpg,86739,1
397,Carmageddon,/r/carmageddon,343,0


In [16]:
subreddits['display_name'] = subreddits['Link'].str.split('/', expand=True)[2]

In [17]:
subreddits['https_Link'] = "https://reddit.com"+subreddits['Link']

In [18]:
subreddits.rename(columns={'Subscribers':'reported_Subscribers', 'Name':'label'}, inplace=True)

In [19]:
subreddits.sample(9)

,label,Link,reported_Subscribers,table_number,display_name,https_Link
403,NeoTokyo,/r/Neotokyo,305,0,Neotokyo,https://reddit.com/r/Neotokyo
415,ZombiU,/r/ZombiU,243,0,ZombiU,https://reddit.com/r/ZombiU
27,Activision,/r/activision,67,6,activision,https://reddit.com/r/activision
532,Ironfell,/r/Ironfell,28,0,Ironfell,https://reddit.com/r/Ironfell
187,Game Commentator Swap,/r/GameCommentatorSwap,10,2,GameCommentatorSwap,https://reddit.com/r/GameCommentatorSwap
457,Rising World,/r/RisingWorld,131,0,RisingWorld,https://reddit.com/r/RisingWorld
456,A Valley Without Wind,/r/AVWW,131,0,AVWW,https://reddit.com/r/AVWW
170,Gaming Tutorials,/r/GGT,51,2,GGT,https://reddit.com/r/GGT
294,Ace of Spades,/r/AceOfSpades,1225,0,AceOfSpades,https://reddit.com/r/AceOfSpades


Inefficient building of fetched subreddit-level metadata..

In [20]:
df = pd.DataFrame()
for display_name in subreddits['display_name']:
    subreddit = reddit.subreddit(display_name)
    try:
        subreddit._fetch()
    except Exception as e:
        print(display_name, e)
    else:
        df = df.append({
            'display_name':subreddit.display_name,
            'subscribers':subreddit.subscribers,
            'name':subreddit.name,
            'id':subreddit.id
        }, ignore_index=True)

CivMulti received 403 HTTP response
eRepublik received 403 HTTP response
planets3 received 403 HTTP response
Chorilion_City_Crimes received 403 HTTP response
Ascend received 403 HTTP response
EverSky received 403 HTTP response
MarkLane received 403 HTTP response
minecraftxe received 403 HTTP response
mncpc received 403 HTTP response
TokyoJungle received 403 HTTP response
Gamingmemories received 403 HTTP response
needateam received 403 HTTP response
gamingartwork received 403 HTTP response
indiejunction received 403 HTTP response
VGracing received 403 HTTP response
pdox received 403 HTTP response
redditcasual received 403 HTTP response
Team_Awesome received 403 HTTP response
rdtclan received 403 HTTP response
wreckedrdt received 403 HTTP response
NAE3 received 403 HTTP response
Sega_Saturn received 403 HTTP response


In [21]:
subreddits = subreddits.merge(df, on='display_name')

In [22]:
subreddits.head()

,label,Link,reported_Subscribers,table_number,display_name,https_Link,id,name,subscribers
0,League of Legends,/r/leagueoflegends,699793,0,leagueoflegends,https://reddit.com/r/leagueoflegends,2rfxx,t5_2rfxx,3251138.0
1,Pokémon,/r/pokemon,444468,0,pokemon,https://reddit.com/r/pokemon,2qmeb,t5_2qmeb,1721593.0
2,Minecraft,/r/Minecraft,440180,0,Minecraft,https://reddit.com/r/Minecraft,2r05i,t5_2r05i,1603649.0
3,The Elder Scrolls V: Skyrim,/r/skyrim,298865,0,skyrim,https://reddit.com/r/skyrim,2s837,t5_2s837,682001.0
4,Hearthstone: Heroes of Warcraft,/r/hearthstone,253622,0,hearthstone,https://reddit.com/r/hearthstone,2w31t,t5_2w31t,1071826.0


testing efficiency...

What other subreddit-level metadata can we fetch?

In [23]:
hi = reddit.subreddit('hellointernet')

In [24]:
dir(hi)

['MESSAGE_PREFIX',
 'STR_FIELD',
 'VALID_TIME_FILTERS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_banned',
 '_comments',
 '_contributor',
 '_create_or_update',
 '_emoji',
 '_fetch',
 '_fetched',
 '_filters',
 '_flair',
 '_info_params',
 '_info_path',
 '_mod',
 '_moderator',
 '_modmail',
 '_muted',
 '_path',
 '_quarantine',
 '_reddit',
 '_reset_attributes',
 '_safely_add_arguments',
 '_stream',
 '_stylesheet',
 '_submission_class',
 '_subreddit_list',
 '_url_parts',
 '_validate_time_filter',
 '_widgets',
 '_wiki',
 'banned',
 'comments',
 'contributor',
 'controversial',
 'display_name',
 'emoji',
 'filters',
 'flair',
 'fullname',
 'gilded',
 'h

### Output subreddits table

In [34]:
subreddits.to_csv("subreddits.csv", index=False)

# Build submissions database

submission-level metadata?

In [ ]:
dir(submission)

In [25]:
lol_top = reddit.subreddit('leagueoflegends').top()

In [26]:
df = pd.DataFrame()
for display_name in subreddits['display_name']:
    for submission in reddit.subreddit(display_name).top(limit=10):
        try:
            df = df.append({
                'subreddit': submission.subreddit,
                'subreddit_id': submission.subreddit_id,
                'title': submission.title,
                'id': submission.id,
                'fullname': submission.fullname,
                'name': submission.name,
                'author_name': submission.author.name, 
                'upvotes': submission.ups,
                'downvotes': submission.downs,
                'score': submission.score, 
                'num_comments': submission.num_comments, 
                'gilded': submission.gilded,
                'domain': submission.domain, 
                'likes': submission.likes,
                'edited': submission.edited,
                'media': submission.media,
                'media_embed': submission.media_embed,
                'media_only': submission.media_only,
                'mod_note': submission.mod_note,
                'author_id': submission.author.id,
                'author_fullname': submission.author.fullname,
                'clicked': submission.clicked,
                'selftext': submission.selftext,
            }, ignore_index=True )
        except Exception as e:
            print(submission.fullname, e)

t3_3uze6n 'Redditor' object has no attribute 'id'
t3_35abkc 'NoneType' object has no attribute 'name'
t3_5lb6zz 'NoneType' object has no attribute 'name'
t3_59u0x5 'NoneType' object has no attribute 'name'
t3_68lpga 'NoneType' object has no attribute 'name'
t3_1tvzp8 'Redditor' object has no attribute 'id'
t3_3by1c2 'NoneType' object has no attribute 'name'
t3_1shl7d 'NoneType' object has no attribute 'name'
t3_52ae7t 'NoneType' object has no attribute 'name'
t3_8lyaam 'NoneType' object has no attribute 'name'
t3_831iqv 'NoneType' object has no attribute 'name'
t3_29t834 'NoneType' object has no attribute 'name'
t3_249uo6 'NoneType' object has no attribute 'name'
t3_5hepn4 'NoneType' object has no attribute 'name'
t3_7wknpo 'NoneType' object has no attribute 'name'
t3_b9tf0r 'Redditor' object has no attribute 'id'
t3_3y4a2b 'NoneType' object has no attribute 'name'
t3_8lx42p 'NoneType' object has no attribute 'name'
t3_1zicl5 'NoneType' object has no attribute 'name'
t3_1l1ldi 'NoneTyp

In [32]:
submissions = df

In [33]:
submissions.sample(9)

,author_fullname,author_id,author_name,clicked,domain,downvotes,edited,fullname,gilded,id,...,media_only,mod_note,name,num_comments,score,selftext,subreddit,subreddit_id,title,upvotes
4523,t2_gldz4,gldz4,vonflare,0.0,self.achron,0.0,0.000000e+00,t3_402s3e,0.0,402s3e,...,0.0,None,t3_402s3e,4.0,7.0,I don't want to buy it if I won't have anyone ...,achron,t5_2sj5z,Is this game dead?,7.0
818,t2_2svr5sdu,2svr5sdu,autumnvirus,0.0,i.redd.it,0.0,0.000000e+00,t3_aczow6,0.0,aczow6,...,0.0,None,t3_aczow6,22.0,265.0,,TeraOnline,t5_2s527,My TERA Online Castanic cosplay! Outfit: Steam...,265.0
3826,t2_ndpfo,ndpfo,hedgepiggywig,0.0,i.redd.it,0.0,0.000000e+00,t3_6evydl,0.0,6evydl,...,0.0,None,t3_6evydl,1.0,25.0,,peggle,t5_2tad5,100% more zen,25.0
455,t2_m4o4j,m4o4j,McJamz,0.0,i.imgur.com,0.0,0.000000e+00,t3_6errck,0.0,6errck,...,0.0,None,t3_6errck,748.0,5617.0,,EliteDangerous,t5_2vi60,WELL THIS JUST HAPPEND,5617.0
2453,t2_b6zpc,b6zpc,Ironshards,0.0,self.mechwarrior,0.0,1.563264e+09,t3_cdtn8j,0.0,cdtn8j,...,0.0,None,t3_cdtn8j,206.0,97.0,Just wanted to keep you folks in the loop with...,mechwarrior,t5_2sr71,"MW5 update: FAQ cut down massively, all mentio...",97.0
1060,t2_4chiu,4chiu,bobabrett,0.0,store.steampowered.com,0.0,0.000000e+00,t3_vozeg,0.0,vozeg,...,0.0,None,t3_vozeg,317.0,511.0,,Tribes,t5_2rdsl,Tribes: Ascend is officially on steam,511.0
3757,t2_9yof7,9yof7,cyberdwarf,0.0,twitter.com,0.0,0.000000e+00,t3_4n6m5g,0.0,4n6m5g,...,0.0,None,t3_4n6m5g,4.0,23.0,,Neotokyo,t5_2r278,Ed Harrison is composing for the next Deus Ex ...,23.0
8053,t2_337xo,337xo,erickhill,0.0,i.redd.it,0.0,0.000000e+00,t3_8fds6p,0.0,8fds6p,...,0.0,None,t3_8fds6p,17.0,122.0,,amiga,t5_2qikh,"Pixel art on the Amiga at its absolute finest,...",122.0
8079,t2_orcvx,orcvx,ShiningConcepts,0.0,github.com,0.0,0.000000e+00,t3_bxsffu,0.0,bxsffu,...,0.0,None,t3_bxsffu,47.0,312.0,,psx,t5_2sayt,So I was doing some Googling and discovered th...,312.0


### Output submmisions table

In [35]:
submissions.to_csv("submissions.csv", index=False)

In [402]:
[(s.title, s.author, s.score, s.id, s.name) for s in hi.top(limit=9)]

[('I like the idea of a 53 stars flag now.',
  Redditor(name='anitussi'),
  2951,
  '6xtapi',
  't3_6xtapi'),
 ('Grey_irl', Redditor(name='Dommkopf_Trip'), 2799, '78tn97', 't3_78tn97'),
 ('Too accurate', Redditor(name='KroniK907'), 2494, '6q2k13', 't3_6q2k13'),
 ('The reality of what Non-Americans understand when Americans tell us their state',
  Redditor(name='ninjomat'),
  2428,
  '9qo5qk',
  't3_9qo5qk'),
 ('CGP Grey. Upvote this so it comes up in a Google image search for CGP Grey.',
  Redditor(name='EarthlyAwakening'),
  2380,
  '6vx6x6',
  't3_6vx6x6'),
 ('the wholesome puppy caretaker',
  Redditor(name='PM_MOI_STEAM_KEYS'),
  1920,
  '7h33mv',
  't3_7h33mv'),
 ("Thought I'd post my Christmas card design mentioned in today's episode :)",
  Redditor(name='squiral-'),
  1848,
  'aclkpi',
  't3_aclkpi'),
 ('Hard as Nails, Cold as Ice',
  Redditor(name='gmalatete'),
  1832,
  'b9gzhv',
  't3_b9gzhv'),
 ('The true unofficial unofficial official Hello Internet street corner',
  Reddito

In [143]:
display_name = 'leagueoflegends'
for submission in reddit.subreddit(display_name).hot(limit=25):
    print((submission.title, submission.author, submission.score, submission.id, submission.name))
    

('Patch 9.15 Discussion and Bug Megathead', Redditor(name='untamedlazyeye'), 45, 'ck5ula', 't3_ck5ula')
('We have added an archive for LCS, LEC, and International Event live discussions', Redditor(name='untamedlazyeye'), 149, 'ckb95m', 't3_ckb95m')
("Twisted Treeline will be removed, but can't we keep Vilemaw?", Redditor(name='Eevree'), 2626, 'ckor77', 't3_ckor77')
('PROJECT: Warwick’s Bloodtrail is exactly the same as the base skin', Redditor(name='parnellyxlol'), 1538, 'ckna65', 't3_ckna65')
('SK Telecom T1 vs. Kingzone DragonX / LCK 2019 Summer - Week 8 / Post-Match Discussion', Redditor(name='adz0r'), 1682, 'ckm739', 't3_ckm739')
('I found an old pentakill edit I made in 2017, I was not on drugs', Redditor(name='hercoule'), 820, 'ckm4uf', 't3_ckm4uf')
("People really does exagerate when they say things like ''Riot is the greediest and scummiest company i've ever seen'' When in reality it's just kind of middle of the pack.", Redditor(name='Sachielkun'), 473, 'cknsi7', 't3_cknsi7')
(

available submission-level metadata:

In [ ]:
dir(submission)

In [104]:
for display_name in subreddits['display_name']:
    subreddit = reddit.subreddit(display_name)
    try:
        new_submissions = [submission.title for submission in subreddit.top(limit=1)]
        submissions['title'] = submissions['title'].append(pd.Series(new_submissions), ignore_index=True)
    except Exception as e:
        print(display_name, e)

CivMulti received 403 HTTP response
eRepublik received 403 HTTP response
planets3 received 403 HTTP response
Chorilion_City_Crimes received 403 HTTP response
Ascend received 403 HTTP response
EverSky received 403 HTTP response
MarkLane received 403 HTTP response
minecraftxe received 403 HTTP response
mncpc received 403 HTTP response
TokyoJungle received 403 HTTP response


In [113]:
submissions['title'] = submissions['title'].append(pd.Series([s.title for s in reddit.subreddit('HelloInternet').top(limit=1)]), ignore_index=True)

In [114]:
submissions

,title
0,I like the idea of a 53 stars flag now.


't3_b7y7k2'